In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Initialize the Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(specs_text)

# Convert text to sequences of integers
sequences = tokenizer.texts_to_sequences(specs_text)

# Pad sequences to make them of equal length (required for neural networks)
max_sequence_length = max(map(len, sequences))
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='post')

# Vocabulary size
vocabulary_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index

In [ ]:
X = pd.DataFrame(padded_sequences)
X.head()

y = to_categorical(clusters)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
embedding_dim = 50
vocab_size = len(tokenizer.word_index) + 1
output_classes = 3

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=X.shape[1]))
model.add(LSTM(100))
model.add(Dense(output_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Model summary
print(model.summary())

In [ ]:
# Train the model
epochs = 25
batch_size = 32
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2)

In [ ]:
accuracy = model.evaluate(X_test, y_test)[1]
print(f'\nTest Accuracy: {accuracy * 100:.2f}%')